In [12]:
# UCSD ECE 143 Programming for Data Analysis
# Winter 2023
# Author: m3cheung@ucsd.edu
# Final Project: create mmr by race bar graph for years 2018-2021

Import modules

In [13]:
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

Load csv files into Pandas DataFrames

In [14]:
races_deaths_df = pd.read_csv(\
    "../data_csv/races-total_deaths.csv").drop(columns=["Notes", "Single Race 6 Code", "Year Code", "Population", "Crude Rate", "Hispanic Origin Code"])
races_births_df = pd.read_csv(\
    "../data_csv/races-total_births.csv").drop(columns=["Notes", "Mother's Single Race 6 Code", "Year Code", "Mother's Hispanic Origin Code"])

display(races_deaths_df)
display(races_births_df)


,Single Race 6,Hispanic Origin,Year,Deaths
0,American Indian or Alaska Native,Not Hispanic or Latino,2019.0,14.0
1,American Indian or Alaska Native,Not Hispanic or Latino,2020.0,13.0
2,American Indian or Alaska Native,Not Hispanic or Latino,2021.0,31.0
3,Asian,Not Hispanic or Latino,2018.0,32.0
4,Asian,Not Hispanic or Latino,2019.0,33.0
...,...,...,...,...
86,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN


,Mother's Single Race 6,Mother's Hispanic Origin,Year,Births
0,American Indian or Alaska Native,Hispanic or Latino,2018.0,7004.0
1,American Indian or Alaska Native,Hispanic or Latino,2019.0,7522.0
2,American Indian or Alaska Native,Hispanic or Latino,2020.0,7950.0
3,American Indian or Alaska Native,Hispanic or Latino,2021.0,8331.0
4,American Indian or Alaska Native,Hispanic or Latino,NaN,30807.0
...,...,...,...,...
119,NaN,NaN,NaN,NaN
120,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN
122,NaN,NaN,NaN,NaN


Clean up data

In [15]:
# remove NaN rows
races_deaths_df = races_deaths_df.dropna()
races_births_df = races_births_df.dropna()

# rename column so that both sets say "Single Race 6"
races_births_df = races_births_df.rename(columns={"Mother's Single Race 6":"Single Race 6"})
# rename column so that both sets say "Hispanic Origin"
races_births_df = races_births_df.rename(
    columns={"Mother's Hispanic Origin":"Hispanic Origin"})

# create new df for the ratio!
ratio_df = races_births_df.merge(races_deaths_df, on="Single Race 6", how="left")

# match years
ratio_df = ratio_df[ratio_df["Year_x"] == ratio_df["Year_y"]]
ratio_df = ratio_df[ratio_df["Hispanic Origin_x"] == ratio_df["Hispanic Origin_y"]]

# isolate hispanic origin rows
hisp_df = ratio_df[ratio_df["Hispanic Origin_x"] == "Hispanic or Latino"]
ratio_df = ratio_df[ratio_df["Hispanic Origin_x"] != "Hispanic or Latino"]

# reset indices to start from 0
ratio_df = ratio_df.reset_index(drop=True)

# add row to include hispanic origin for year 2018
ratio_df.loc[len(ratio_df.index)] = ["Hispanic Origin", "", "2018", hisp_df[hisp_df["Year_x"] == 2018.0]["Births"].sum(), "", "", hisp_df[hisp_df["Year_x"] == 2018.0]["Deaths"].sum()]

# add row to include hispanic origin for year 2019
ratio_df.loc[len(ratio_df.index)] = ["Hispanic Origin", "", "2019", hisp_df[hisp_df["Year_x"] == 2019.0]["Births"].sum(), "", "", hisp_df[hisp_df["Year_x"] == 2019.0]["Deaths"].sum()]

# add row to include hispanic origin for year 2020
ratio_df.loc[len(ratio_df.index)] = ["Hispanic Origin", "", "2020", hisp_df[hisp_df["Year_x"] == 2020.0]["Births"].sum(), "", "", hisp_df[hisp_df["Year_x"] == 2020.0]["Deaths"].sum()]

# add row to include hispanic origin for year 2021
ratio_df.loc[len(ratio_df.index)] = ["Hispanic Origin", "", "2021", hisp_df[hisp_df["Year_x"] == 2021.0]["Births"].sum(), "", "", hisp_df[hisp_df["Year_x"] == 2021.0]["Deaths"].sum()]

# drop all extra columns
ratio_df = ratio_df.drop(columns=["Hispanic Origin_x", "Hispanic Origin_y", "Year_y"])

# Rename Year column
ratio_df = ratio_df.rename(columns={"Year_x":"Year"})

# remove more than one race rows, due to data labeled as "unreliable"
ratio_df = ratio_df.drop(ratio_df[ratio_df["Single Race 6"] == "More than one race"].index, axis=0)
# ratio_df = ratio_df.drop(ratio_df[ratio_df["Single Race 6"] == "American Indian or Alaska Native"].index, axis=0)

# consider suppressed rows as 0, since suppressed < 10
ratio_df["Deaths"] = ratio_df["Deaths"].fillna(0)

display(ratio_df)

,Single Race 6,Year,Births,Deaths
0,American Indian or Alaska Native,2019.0,28450.0,14.0
1,American Indian or Alaska Native,2020.0,26813.0,13.0
2,American Indian or Alaska Native,2021.0,26124.0,31.0
3,Asian,2018.0,240798.0,32.0
4,Asian,2019.0,238769.0,33.0
5,Asian,2020.0,219068.0,27.0
6,Asian,2021.0,213813.0,36.0
7,Black or African American,2018.0,552029.0,206.0
8,Black or African American,2019.0,548075.0,241.0
9,Black or African American,2020.0,529811.0,293.0


Calculate MMR for each row

In [16]:
# create column for MMR = (Deaths/Live Births * 100,000)
ratio_df["MMR per 100,000 Live Births"] = round(
    ratio_df["Deaths"] / ratio_df["Births"] * 100000)

display(ratio_df)

,Single Race 6,Year,Births,Deaths,"MMR per 100,000 Live Births"
0,American Indian or Alaska Native,2019.0,28450.0,14.0,49.0
1,American Indian or Alaska Native,2020.0,26813.0,13.0,48.0
2,American Indian or Alaska Native,2021.0,26124.0,31.0,119.0
3,Asian,2018.0,240798.0,32.0,13.0
4,Asian,2019.0,238769.0,33.0,14.0
5,Asian,2020.0,219068.0,27.0,12.0
6,Asian,2021.0,213813.0,36.0,17.0
7,Black or African American,2018.0,552029.0,206.0,37.0
8,Black or African American,2019.0,548075.0,241.0,44.0
9,Black or African American,2020.0,529811.0,293.0,55.0


Create Map

In [17]:
fig = px.histogram(ratio_df, x="Year", y="MMR per 100,000 Live Births",
                   color='Single Race 6', barmode='group',
                   category_orders={"Single Race 6":["More than one race", "Asian", "Hispanic Origin", "White", "Black or African American", "American Indian or Alaska Native"]},
                   color_discrete_sequence=px.colors.qualitative.G10,
                   height=400, width=800)
fig.show()
# should I include all the races even if there is a year missing?
# should I label each bar individually?
# should I include AINA group because their crude rates were labeled unreliable for years before 2021